In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets
import plotly.express as px

from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_recall_curve, average_precision_score, \
    roc_auc_score, roc_curve, auc, recall_score, precision_score
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

import seaborn as sns
import scipy.stats
from scipy import stats
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from xgboost import XGBClassifier

import time

pd.set_option("display.max_columns", None)

database = pd.read_csv('../data/weatherAUS.csv')

In [4]:
targetNulls = widgets.ToggleButtons(
    options=['No', 'Yes', 'Delete'],
    description='Optios:',
    disabled=False,
    tooltips=['No', 'Yes', 'Delete']
)
print("Tractament del target (nulls)")
targetNulls

Tractament del target (nulls)


ToggleButtons(description='Optios:', options=('No', 'Yes', 'Delete'), tooltips=('No', 'Yes', 'Delete'), value=…

In [5]:
balance = widgets.ToggleButtons(
    options=['Oversampling', 'Oversampling Smote' ,'Subsampling','None'],
    description='Optios:',
    disabled=False,
    tooltips=['Oversampling', 'Oversampling Smote' ,'Subsampling', 'None']
)
print("Balanceig de les dades")
balance

Balanceig de les dades


ToggleButtons(description='Optios:', options=('Oversampling', 'Oversampling Smote', 'Subsampling', 'None'), to…

In [6]:
atributeNulls = widgets.ToggleButtons(
    options=['Mode', 'Median', 'Mean'],
    description='Optios:',
    disabled=False,
    tooltips=['Mode', 'Median', 'Mean']
)
print("Tractament de les dades continues")
display(atributeNulls)

Tractament de les dades continues


ToggleButtons(description='Optios:', options=('Mode', 'Median', 'Mean'), tooltips=('Mode', 'Median', 'Mean'), …

In [7]:
if(targetNulls.value == 'No'):
    database['RainTomorrow'] = [1 if i=='Yes' else 0 for i in database['RainTomorrow'] ]
elif(targetNulls.value == 'Yes'):
    database['RainTomorrow']=[0 if i=='No' else 1 for i in database['RainTomorrow'] ]
else:
    database = database.dropna(axis=0, subset=['RainTomorrow'])
    
cols_to_drop=['Date']
database.drop(columns=cols_to_drop,inplace=True)

x=database.drop(['RainTomorrow'],axis=1)
y=database['RainTomorrow']

count_class_0, count_class_1 = database.RainTomorrow.value_counts()
if(balance.value == 'Oversampling'):
    df_class_0 = database[database['RainTomorrow'] == "No"]
    df_class_1 = database[database['RainTomorrow'] == "Yes"]
    
    df_class_1_over = df_class_1.sample(count_class_0, replace=True)
    database = pd.concat([df_class_0, df_class_1_over], axis=0)
    
elif(balance.value == 'Subsampling'):
    df_class_0 = database[database['RainTomorrow'] == "No"]
    df_class_1 = database[database['RainTomorrow'] == "Yes"]

    df_class_0_under = df_class_0.sample(count_class_1)
    database = pd.concat([df_class_0_under, df_class_1], axis=0)

database['RainTomorrow'] = [1 if i=='Yes' else 0 for i in database['RainTomorrow'] ]

categorical_cols = list(x.select_dtypes(include=['object']).columns)
continuous_cols = list(x.select_dtypes(include=['float64']).columns)

for i in categorical_cols:
    x[i].fillna(x[i].mode()[0], inplace=True)

if(targetNulls.value == 'Mode'):
    for column in continuous_cols:
        x[column].fillna(x[column].mode()[0], inplace = True)

elif(targetNulls.value == 'Median'):

    for column in continuous_cols:
        x[column].fillna(x[column].median(), inplace = True)
else:
    for column in continuous_cols:
        x[column].fillna(x[column].mean(), inplace = True)

transformer = ColumnTransformer(transformers=[('ohe', OneHotEncoder(sparse='False',drop='first'), categorical_cols)],remainder='passthrough')
x=transformer.fit_transform(x)

if balance.value == 'Oversampling Smote':
    smote=SMOTE()
    x,y=smote.fit_resample(x,y)

    print(x.shape,y.shape)
    
x=pd.DataFrame(x.toarray())

def check_skewness(x):
    
    skew_limit=0.75
    skew_value=x[x.columns].skew()
    skew_cols=skew_value[abs(skew_value)>skew_limit]
    #print(skew_cols)
    cols=skew_cols.index
    return cols
    
skewed_col=check_skewness(x)
pt=PowerTransformer(standardize=False)
x[skewed_col]=pt.fit_transform(x[skewed_col])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
sc=StandardScaler(with_mean=False)
sc=sc.fit(X_train)
X_train=sc.transform(X_train)
X_test=sc.transform(X_test)

print("Opcio,s escollides:")
print("--------------------------------------")
print("Target NUllS :", targetNulls.value)
print("Balance :", balance.value)
print("Dades continues Nulls:", atributeNulls.value)



rf=RandomForestClassifier(n_estimators= 50, max_samples= 0.3, max_features='log2', criterion= 'gini', bootstrap= False)
rf.fit(X_train,y_train)
print("Model entrenat")

ValueError: a must be greater than 0 unless no samples are taken